In [ ]:
import pandas as pd
from torch.utils.data import DataLoader
from transformer import LinearData, Transformer, LitTransformer

# use the data from ../generate
samples_df = pd.read_csv("../data/blooms.csv")
dataset = LinearData(samples_df)
loader = DataLoader(dataset, batch_size = 64)

In [ ]:
from captum.attr import IntegratedGradients

checkpoint = "lightning_logs/version_0/checkpoints/epoch=39-step=1280.ckpt"
lit_model = LitTransformer.load_from_checkpoint(checkpoint, model = Transformer())

def forward(model):
  return lambda x: model(x)[1]

ig = IntegratedGradients(forward(lit_model.model))
x, y = next(iter(loader))
i_hat = ig.attribute(x)

In [9]:
import numpy as np
i_hat.shape
np.savetxt("../data/attributions.csv", np.reshape(i_hat, (64, -1)), delimiter=',')